In [1]:
#@title Run this cell to mount Google Drive and get `kaggle.json` from personal directory

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from pathlib import Path

# TEMPORARY CHANGE
DATASET_DIR = Path("./drive/My Drive/bdb-2025/working/datasets/")

# Step: Model Implementation



In [3]:
!wget https://raw.githubusercontent.com/illydh/2025-nfl-bdb/refs/heads/main/scripts/process_datasets.py

--2024-12-13 02:00:44--  https://raw.githubusercontent.com/illydh/2025-nfl-bdb/refs/heads/main/scripts/process_datasets.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9647 (9.4K) [text/plain]
Saving to: ‘process_datasets.py’

process_datasets.py 100%[===================>]   9.42K  --.-KB/s    in 0s      

2024-12-13 02:00:44 (85.4 MB/s) - ‘process_datasets.py’ saved [9647/9647]



## Load Datasets

In [4]:
import torch
from torch.utils.data import DataLoader
from process_datasets import load_datasets
from process_datasets import BDB2025_Dataset

# Load preprocessed datasets
train_dataset = load_datasets(model_type='transformer', split='train')
val_dataset = load_datasets(model_type='transformer', split='val')
test_dataset = load_datasets(model_type='transformer', split='test')

In [5]:
# Create DataLoader objects
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=3)
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=3)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=3)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
# Print feature and target shapes from DataLoaders
for batch in train_loader:
    features, targets = batch
    print("Train features shape:", features.shape)
    print("Train targets shape:", targets.shape)
    break

Train features shape: torch.Size([64, 21, 21])
Train targets shape: torch.Size([64, 21])


## Load Model

In [ ]:
!wget https://raw.githubusercontent.com/illydh/2025-nfl-bdb/refs/heads/main/scripts/models.py

--2024-12-12 05:32:30--  https://raw.githubusercontent.com/illydh/2025-nfl-bdb/refs/heads/main/scripts/models.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8927 (8.7K) [text/plain]
Saving to: ‘models.py.1’

models.py.1         100%[===================>]   8.72K  --.-KB/s    in 0s      

2024-12-12 05:32:30 (72.7 MB/s) - ‘models.py.1’ saved [8927/8927]



In [7]:
!pip install pytorch-lightning  #  module not included in colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 41.9 MB/s eta 0:00:00


In [19]:
#import models
#from models import SportsTransformer
#from models import SportsTransformerLitModel

"""
Model Architectures for NFL Big Data Bowl 2025 prediction task

This module defines neural network architectures for predicting ouptuts in NFL Plays. It includes a generalized transformer-based model for
processing sports tracking data and a LightningModule wrapper for training and evaluation.

Classes:
    SportsTransformer: Generalized Transformer-based model for sports tracking data.
    SportsTransformerLitModel: LightningModule wrapper for shared training functionality.
"""

from typing import Any
import torch
from torch import Tensor, nn, squeeze
from torch.optim import AdamW
from pytorch_lightning import LightningModule

torch.set_float32_matmul_precision("medium")

class SportsTransformer(nn.Module):
    """
    Transformer model architecture for processing sports tracking data.

    This model leverages self-attention mechanisms to process player tracking data, capturing
    the spatial and temporal relationships between players on the field.

    Attributes:
        feature_len (int): Number of input features per player.
        model_dim (int): Dimension of the model's internal representations.
        num_layers (int): Number of transformer encoder layers.
        dropout (float): Dropout rate for regularization.
        hyperparams (dict): Dictionary storing hyperparameters of the model.
    """

    def __init__(
        self,
        feature_len: int,
        model_dim: int = 32,
        num_layers: int = 2,
        output_dim: int = 7,
        dropout: float = 0.3,
    ):
        """
        Initialize the SportsTransformer.

        Args:
            feature_len (int): Number of input features per player.
            model_dim (int): Dimension of the model's internal representations.
            num_layers (int): Number of transformer encoder layers.
            dropout (float): Dropout rate for regularization.
        """
        super().__init__()
        dim_feedforward = model_dim * 4
        num_heads = min(16, max(2, 2 * round(model_dim / 64)))  # Attention is optimized for even number of heads

        self.hyperparams = {
            "model_dim": model_dim,
            "num_layers": num_layers,
            "num_heads": num_heads,
            "dim_feedforward": dim_feedforward,
        }

        # Normalize input features
        self.feature_norm_layer = nn.BatchNorm1d(feature_len)

        # Embed input features to model dimension
        self.feature_embedding_layer = nn.Sequential(
            nn.Linear(feature_len, model_dim),
            nn.ReLU(),
            nn.LayerNorm(model_dim),
            nn.Dropout(dropout),
        )

        # Transformer Encoder
        # This component applies multiple layers of self-attention and feed-forward networks
        # to process player data in a permutation-equivariant manner.
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=model_dim,
                nhead=num_heads,
                dim_feedforward=dim_feedforward,
                dropout=dropout,
                batch_first=True,
            ),
            num_layers=num_layers,
        )

        # Pool across player dimension
        self.player_pooling_layer = nn.AdaptiveAvgPool1d(1)

        # Task-specific Decoder to predict output.
        self.decoder = nn.Sequential(
            nn.Linear(model_dim, model_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(model_dim, model_dim // 4),
            nn.ReLU(),
            nn.LayerNorm(model_dim // 4),
            nn.Linear(model_dim // 4, output_dim),  # Adjusted to match target shape
        )

    def forward(self, x: Tensor) -> Tensor:
        """
        Forward pass of the SportsTransformer.

        Args:
            x (Tensor): Input tensor of shape [batch_size, num_players, feature_len].

        Returns:
            Tensor: Predicted output of shape [batch_size, 22].
        """
        # x: [B: batch_size, P: # of players, F: feature_len]
        B, P, F = x.size()

        # Normalize features
        x = self.feature_norm_layer(x.permute(0, 2, 1)).permute(0, 2, 1)  # [B,P,F] -> [B,P,F]

        # Embed features
        x = self.feature_embedding_layer(x)  # [B,P,F] -> [B,P,M: model_dim]

        # Apply transformer encoder
        x = self.transformer_encoder(x)  # [B,P,M] -> [B,P,M]

        # Pool over player dimension
        x = squeeze(self.player_pooling_layer(x.permute(0, 2, 1)), -1)  # [B,M,P] -> [B,M]

        # Decode to predict output
        x = self.decoder(x)  # [B,M] -> [B, output_dim]

        return x

class SportsTransformerLitModel(LightningModule):
    """
    Lightning module for training and evaluating models.

    This class wraps the SportsTransformer for training, validation, and testing
    using PyTorch Lightning, providing a structured training loop, optimization, and logging.

    Attributes:
        feature_len (int): Number of input features per player.
        batch_size (int): Batch size for training and evaluation.
        model (SportsTransformer): The transformer-based model.
        learning_rate (float): Learning rate for the optimizer.
        loss_fn (nn.Module): Loss function used for training.
    """

    def __init__(
        self,
        feature_len: int,
        batch_size: int,
        model_dim: int,
        num_layers: int,
        output_dim: int,
        dropout: float = 0.1,
        learning_rate: float = 1e-3,
    ):
        """
        Initialize the SportsTransformerLitModel.

        Args:
            feature_len (int): Number of input features.
            batch_size (int): Batch size for training and evaluation.
            model_dim (int): Dimension of the model's internal representations.
            num_layers (int): Number of layers in the model.
            dropout (float): Dropout rate for regularization.
            learning_rate (float): Learning rate for the optimizer.
        """
        super().__init__()
        self.feature_len = feature_len
        self.model = SportsTransformer(
            feature_len=self.feature_len, model_dim=model_dim, num_layers=num_layers, dropout=dropout, output_dim=output_dim
        )
        self.example_input_array = torch.randn((batch_size, 21, self.feature_len))  # changed to 21

        self.learning_rate = learning_rate
        self.num_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        self.hparams["params"] = self.num_params
        for k, v in self.model.hyperparams.items():
            self.hparams[k] = v

        self.save_hyperparameters()
        self.loss_fn = torch.nn.CrossEntropyLoss()
        # self.loss_fn = torch.nn.BCEWithLogitsLoss()

    def forward(self, x: Tensor) -> Tensor:
        """
        Forward pass of the model.

        Args:
            x (Tensor): Input tensor.

        Returns:
            Tensor: Output tensor.
        """
        if isinstance(x, list):
            x = torch.stack(x)
        return self.model(x)

    def training_step(self, batch: tuple[Tensor, Tensor], batch_idx: int) -> Tensor:
        """
        Perform a single training step.

        Args:
            batch (tuple[Tensor, Tensor]): Batch of input features and target locations.
            batch_idx (int): Index of the current batch.

        Returns:
            Tensor: Computed loss for the batch.
        """
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        return loss

    def validation_step(self, batch: tuple[Tensor, Tensor], batch_idx: int) -> Tensor:
        """
        Validation step for the model.

        Args:
            batch (tuple[Tensor, Tensor]): Batch of input and target tensors.
            batch_idx (int): Index of the current batch.

        Returns:
            Tensor: Computed loss.
        """
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss_fn(y_hat, y)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        return loss

    def predict_step(self, batch: tuple[Tensor, Tensor], batch_idx: int, dataloader_idx: int = 0) -> Tensor:
        """
        Prediction step for the model.

        Args:
            batch (tuple[Tensor, Tensor]): Batch of input and target tensors.
            batch_idx (int): Index of the current batch.
            dataloader_idx (int): Index of the dataloader.

        Returns:
            Tensor: Predicted output tensor.
        """
        x, _ = batch
        if isinstance(x, list):
            x = torch.stack(x)
        y_hat = self.model(x)
        return y_hat

    def configure_optimizers(self) -> AdamW:
        """
        Configure the optimizer for training.

        Returns:
            AdamW: Configured optimizer.
        """
        return AdamW(self.parameters(), lr=self.learning_rate)

In [20]:
# Model parameters
feature_len = 21  # Adjust this as needed based on input data
model_dim = 64  # Dimension of transformer model (adjustable)
num_layers = 12  # Number of transformer layers (adjustable)
dropout = 0.01
learning_rate = 1e-3
batch_size = 64   #   adjust when analyzing all tracking information
output_dim = 21
epochs = 40

# Initialize the model
model = SportsTransformerLitModel(
    feature_len=feature_len,
    batch_size=batch_size,
    model_dim=model_dim,
    num_layers=num_layers,
    output_dim=output_dim,
    dropout=dropout,
    learning_rate=learning_rate,
)

In [21]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pathlib import Path

# Define checkpointing to save the best model
checkpoint_callback = ModelCheckpoint(
   dirpath=Path("./drive/My Drive/bdb-2025/"),      # adjustable dir path
   filename="best-checkpoint",
   save_top_k=1,
   verbose=True,
   monitor="val_loss",
   mode="min",
)

# Define early stopping
early_stop_callback = EarlyStopping(
   monitor="val_loss",
   min_delta=0.01,  # Minimum change in monitored value to qualify as an improvement
   patience=3,      # Number of epochs with no improvement after which training will be stopped
   verbose=True,
   mode="min"
)

# Initialize the trainer
trainer = Trainer(
   max_epochs=epochs,  # Adjust the number of epochs
   accelerator="gpu",  # Use 'gpu' if CUDA is available, otherwise use 'cpu'
   devices=1,
   callbacks=[checkpoint_callback, early_stop_callback],
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [22]:
# Start training
trainer.fit(model, train_loader, val_loader)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/drive/My Drive/bdb-2025 exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type              | Params | Mode  | In sizes     | Out sizes
---------------------------------------------------------------------------------
0 | model   | SportsTransformer | 606 K  | train | [64, 21, 21] | [64, 21] 
1 | loss_fn | CrossEntropyLoss  | 0      | train | ?            | ?        
---------------------------------------------------------------------------------
606 K     Trainable params
0         Non-trainable params
606 K     Total params
2.428     Total estimated model params size (MB)
139       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 12416: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 15520: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Monitored metric val_loss did not improve in the last 3 records. Best score: 59.256. Signaling Trainer to stop.
INFO:pytorch_lightning.utilities.rank_zero:Epoch 5, global step 18624: 'val_loss' was not in top 1


In [23]:
# Inference on test data
predictions = trainer.predict(model, test_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [24]:
# Concatenate predictions into a single tensor
predictions_tensor = torch.cat(predictions, dim=0)

In [25]:
# Assuming predictions are logits for a multi-class problem
predicted_labels = torch.argmax(predictions_tensor, dim=1)

In [26]:
import numpy as np

# Extract true labels from the test_loader
y_true = torch.cat([y for _, y in test_loader], dim=0)

# Convert tensors to numpy arrays if needed for sklearn functions
y_true_np = np.argmax(y_true.cpu().numpy(), axis=-1)
predicted_labels_np = predicted_labels.cpu().numpy()

print("y_true shape:", y_true_np.shape)
print("Predicted labels shape:", predicted_labels_np.shape)

y_true shape: (42793,)
Predicted labels shape: (42793,)


In [27]:
import pandas as pd

# Create a test dataframe
df_test = pd.DataFrame({
    'gameId': [key[0] for key in test_dataset.keys],
    'playId': [key[1] for key in test_dataset.keys],
    'frameId': [key[2] for key in test_dataset.keys],
    'true_labels': y_true_np,
    'predicted_labels': predicted_labels_np
})

In [35]:
test_dir = Path("./drive/My Drive/bdb-2025/split_prepped_data/test_features.parquet")

# Attach metadata and filter to ball_snap event only
df_test_metadata = pd.read_parquet(test_dir)

df_test = df_test.merge(df_test_metadata[["gameId", "playId", "frameId", "frameType"]], on=["gameId", "playId", "frameId"], how="left")

In [36]:
# Remove frame after the snap
df_test_before_snap = df_test[df_test.frameType == "BEFORE_SNAP"]

In [39]:
# Filter to ball_snap event for evaluation
df_test_ball_snap = df_test[df_test.frameType == "SNAP"]

In [40]:
df_test_ball_snap = df_test_ball_snap.drop_duplicates(subset=['gameId', 'playId', 'frameId'])

df_test_ball_snap = df_test_ball_snap.sort_values(['gameId', 'playId', 'frameId']).reset_index(drop=True)

display(df_test_ball_snap)

,gameId,playId,frameId,true_labels,predicted_labels,frameType
0,2022090800,80,88,19,19,SNAP
1,2022090800,122,113,19,19,SNAP
2,2022090800,167,102,19,19,SNAP
3,2022090800,933,129,19,19,SNAP
4,2022090800,1009,112,19,19,SNAP
...,...,...,...,...,...,...
264,2022091200,3048,142,19,19,SNAP
265,2022091200,3194,182,19,19,SNAP
266,2022091200,3245,77,20,20,SNAP
267,2022091200,3404,100,20,20,SNAP


In [41]:
true_labels = df_test_ball_snap['true_labels'].values
predicted_labels = df_test_ball_snap['predicted_labels'].values